# Installation & Configuration

In [ ]:
!pip install kaggle
!pip install tqdm

import os
import torch
import pandas as pd
import torch.nn as nn
import numpy as np
import torchvision.transforms.v2 as transforms
import matplotlib.pyplot as plt
import cv2

from PIL import Image
from tqdm import tqdm
from google.colab import files
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from google.colab import drive
from sklearn.model_selection import train_test_split
from torchvision.models import resnet50, ResNet50_Weights


drive.mount('/content/drive')
kaggle_json_path = '/content/drive/MyDrive/ColabNotebooks/A5/kaggle.json'

# Copy kaggle.json to the correct location
!mkdir -p ~/.kaggle
!cp {kaggle_json_path} ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Choose the model you want to apply
model_type = "resnet50" # or "inceptionv3"

# Data Setup

In [ ]:
# Download and unzip dataset
!kaggle datasets download -d rezaunderfit/48k-imdb-movies-with-posters > /dev/null 2>&1
!unzip -q 48k-imdb-movies-with-posters.zip

# Load title basics
tsv_path = '/content/drive/MyDrive/ColabNotebooks/A5/title.basics.tsv'
title_basics = pd.read_csv(tsv_path, sep='\t', na_values='\\N')

# List all files in the Poster directory
poster_dir = 'Poster'
poster_files = []
for root, _, files in os.walk(poster_dir):
    for file in files:
        if file.endswith('.jpg'):
            file_path = os.path.join(root, file)
            if os.path.getsize(file_path) > 0:  # Only include non-zero byte files
                poster_files.append(file_path)

# Extract tconst and startYear from file paths
poster_info = []
for file_path in poster_files:
    parts = file_path.split('/')
    start_year = parts[1]
    tconst = parts[2]
    poster_info.append((start_year, tconst))

# Convert to DataFrame
poster_df = pd.DataFrame(poster_info, columns=['startYear', 'tconst'])

# Ensure startYear is an integer
poster_df['startYear'] = poster_df['startYear'].astype(int)
title_basics['startYear'] = title_basics['startYear'].astype(float).fillna(0).astype(int)  # Handle missing startYear and convert to int

# Filter movies from the past 50 years
current_year = 2024
past_50_years = current_year - 50
poster_df = poster_df[poster_df['startYear'] >= past_50_years]

# Merge with title_basics to keep only relevant records
title_basics_filtered = pd.merge(title_basics, poster_df, on=['startYear', 'tconst'])

# Function to count genres
def count_genres(metadata):
    genre_counter = Counter()
    for genres in metadata['genres'].dropna():
        first_genre = genres.split(',')[0]
        genre_counter[first_genre] += 1
    return genre_counter

# Count genres in the filtered dataset
filtered_genre_counts = count_genres(title_basics_filtered)

# Calculate total number of movies
total_movies = len(title_basics_filtered)

# Filter out genres with less than 1% of the total dataset
min_count = total_movies * 0.01
valid_genres = {genre for genre, count in filtered_genre_counts.items() if count >= min_count}

# Filter the dataset to only include valid genres
def filter_genres(row):
    if pd.notna(row['genres']):
        genres = row['genres'].split(',')
        if any(genre in valid_genres for genre in genres):
            return True
    return False

title_basics_filtered = title_basics_filtered[title_basics_filtered.apply(filter_genres, axis=1)]

# Create your data splits
train_metadata, test_metadata = train_test_split(title_basics_filtered, test_size=0.2, random_state=42)
train_metadata, val_metadata = train_test_split(train_metadata, test_size=0.25, random_state=42)
print(f"Train size: {len(train_metadata)}, Validation size: {len(val_metadata)}, Test size: {len(test_metadata)}")

# Count genres in the training dataset
train_genre_counts = count_genres(train_metadata)

# Total number of movies in the training dataset
train_total_movies = len(train_metadata)

# Calculate and print genre distribution with percentages
print("\nGenre Distribution in Training Dataset:")
for genre, count in train_genre_counts.items():
    percentage = (count / train_total_movies) * 100
    print(f"{genre} - {count} ({percentage:.2f}%)")

# Define the image transformations
if model_type == "inceptionv3":
  image_transforms = transforms.Compose([
      transforms.Resize(299),
      transforms.CenterCrop(299),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])
elif model_type == "resnet50":
  image_transforms = ResNet50_Weights.IMAGENET1K_V2.transforms()

class MovieDataset(Dataset):
    def __init__(self, metadata, img_dir, transform=None, genre_to_index=None):
        self.metadata = metadata
        self.img_dir = img_dir
        self.transform = transform
        self.genre_to_index = genre_to_index

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        tconst = self.metadata.iloc[idx]['tconst']
        start_year = self.metadata.iloc[idx]['startYear']
        img_name = os.path.join(self.img_dir, str(start_year), tconst, f"{tconst}.jpg")
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        genres = self.metadata.iloc[idx]['genres']
        genre_tensor = self.genres_to_tensor(genres)
        return image, genre_tensor

    def genres_to_tensor(self, genres):
        first_genre = genres.split(',')[0] if pd.notna(genres) else 'Unknown'
        genre_index = self.genre_to_index.get(first_genre, self.genre_to_index['Unknown'])
        return torch.tensor(genre_index, dtype=torch.long)

# Create a mapping from genre to index based on the filtered dataset
filtered_genres = set(g.split(',')[0] for g in title_basics_filtered['genres'].dropna())
genre_to_index = {genre: idx for idx, genre in enumerate(filtered_genres)}

# Ensure 'Unknown' genre is included in genre_to_index
genre_to_index['Unknown'] = len(genre_to_index)

# Directory containing images
img_dir = 'Poster'

# Create datasets
train_dataset = MovieDataset(metadata=train_metadata, img_dir=img_dir, transform=image_transforms, genre_to_index=genre_to_index)
val_dataset = MovieDataset(metadata=val_metadata, img_dir=img_dir, transform=image_transforms, genre_to_index=genre_to_index)
test_dataset = MovieDataset(metadata=test_metadata, img_dir=img_dir, transform=image_transforms, genre_to_index=genre_to_index)

# Model Setup

In [ ]:
batch_size = 32

# data loader
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=2
)

# Set up model for fine tuning
if model_type == "inceptionv3":
  model = torch.hub.load("pytorch/vision:v0.10.0", "inception_v3", pretrained=True)
  model.aux_logits = False  # Disable auxiliary logits
  num_genres = len(genre_to_index)
  model.fc = nn.Linear(model.fc.in_features, num_genres)  # Adjust the final layer
  path = "/content/drive/MyDrive/ColabNotebooks/A5/best_model_inception.pth"
elif model_type == "resnet50":
  model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
  num_genres = len(genre_to_index)
  model.fc = nn.Linear(model.fc.in_features, num_genres)  # Adjust the final layer
  path = "/content/drive/MyDrive/ColabNotebooks/A5/best_model_resnet50.pth"

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set up loss function
criterion = nn.CrossEntropyLoss()

# Load the best model weights
model.load_state_dict(
    torch.load(path)
)

# Evaluate on test set and store the best predictions per genre
model.eval()
best_predictions = {genre: {"confidence": 0, "image_path": None, "tconst": None} for genre in genre_to_index.keys()}

with torch.no_grad():
    total = 0
    test_loader_tqdm = tqdm(test_loader, desc="Testing", unit="batch")
    for test_images, test_genres in test_loader_tqdm:
        test_images = test_images.to(device)
        test_genres = test_genres.to(device)
        test_outputs = model(test_images)

        for idx in range(test_images.size(0)):
            genre = test_genres[idx].item()
            confidence = torch.nn.functional.softmax(test_outputs[idx], dim=0)[genre].item()
            row = title_basics_filtered.iloc[total + idx]
            image_path = row["Poster"]
            tconst = row["tconst"]

            if confidence > best_predictions[genre]["confidence"]:
                best_predictions[genre]["confidence"] = confidence
                best_predictions[genre]["image_path"] = image_path
                best_predictions[genre]["tconst"] = tconst

        total += test_genres.size(0)


# GradCAM Analysis

In [ ]:
# Prepare the Image
def preprocess_image(img_path):
    if model_type == "inceptionv3":
      preprocess = transforms.Compose(
          [
              transforms.Resize((299, 299)),
              transforms.ToTensor(),
              transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
          ]
      )
    elif model_type == "resnet50":
      preprocess = ResNet50_Weights.IMAGENET1K_V2.transforms()

    img = Image.open(img_path).convert("RGB")
    img_tensor = preprocess(img).unsqueeze(0)  # Add batch dimension
    return img_tensor

# Get the Gradients and Activations
def get_gradients_hook(module, grad_input, grad_output):
    gradients.append(grad_output[0])

def get_activations_hook(module, input, output):
    activations.append(output)

gradients = []
activations = []

target_layer = model.Mixed_7c
target_layer.register_forward_hook(get_activations_hook)
target_layer.register_backward_hook(get_gradients_hook)

# Compute Grad-CAM
def compute_gradcam(activations, gradients):
    pooled_gradients = torch.mean(gradients[0], dim=[0, 2, 3])
    activations = activations[0].squeeze(0).detach()  # Remove batch dimension

    for i in range(len(pooled_gradients)):
        activations[i, :, :] *= pooled_gradients[i]

    heatmap = torch.mean(activations, dim=0).cpu()
    heatmap = np.maximum(heatmap, 0)
    heatmap /= torch.max(heatmap)

    return heatmap

# Visualize the Heatmap side by side with original image
def visualize_heatmap(img_path, heatmap, tconst, confidence):
    original_img = cv2.imread(img_path)
    heatmap_resized = cv2.resize(heatmap.numpy(), (original_img.shape[1], original_img.shape[0]))
    heatmap_resized = np.uint8(255 * heatmap_resized)
    heatmap_resized = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(original_img, 0.5, heatmap_resized, 0.5, 0)

    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    ax[0].axis("off")
    ax[0].set_title("Original Image")

    ax[1].imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
    ax[1].axis("off")
    ax[1].set_title("Grad-CAM Heatmap")

    plt.suptitle(f"tconst: {tconst}, Confidence: {confidence:.4f}")
    plt.show()

# Function to generate Grad-CAM heatmap
def generate_gradcam_heatmap(image_path, model):
    input_tensor = preprocess_image(image_path)

    gradients.clear()
    activations.clear()

    output = model(input_tensor)
    pred_class = output.argmax().item()

    model.zero_grad()
    one_hot_output = torch.zeros((1, output.size()[-1]), dtype=torch.float32)
    one_hot_output[0][pred_class] = 1
    output.backward(gradient=one_hot_output)

    heatmap = compute_gradcam(activations, gradients)
    return heatmap

# Process each genre and generate heatmap
for genre, data in best_predictions.items():
    image_path = data["image_path"]
    tconst = data["tconst"]
    confidence = data["confidence"]
    if image_path is not None:
        print(f"Generating Grad-CAM for genre: {genre}")
        heatmap = generate_gradcam_heatmap(image_path, model)
        visualize_heatmap(image_path, heatmap, tconst, confidence)
    else:
        print(f"No predictions found for genre: {genre}")